In [7]:
from datasets import load_dataset, Audio
import io
import soundfile as sf
import numpy as np

from datasets import load_dataset, Features, Value, Array2D, Array3D
from torch.utils.data import DataLoader as TorchLoader
import io
import soundfile as sf
import torch

import torch.nn.functional as F
from transformers import WhisperProcessor, WhisperForConditionalGeneration

from IPython.display import Audio, display, HTML

So the main issue was that the hf decoder was not working because there was some issue with torchcodec. By disabling automatic decoding and manually reading the raw bytes, we can use soundfile or librosa to decode the audio data correctly.

Preprocessing classes -> HFDataweapper and ASRPipeline
fn -> collate_batch

In [2]:
class HFDataWrapper:
    def __init__(self, ds_split):
        self.ds = ds_split

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        sample = self.ds[idx]
        audio_info = sample["audio"]

        # manually decode bytes using soundfile
        raw_bytes = audio_info["bytes"]
        with io.BytesIO(raw_bytes) as f:
            waveform, sr = sf.read(f, dtype="float32")

        # convert to torch tensor
        if waveform.ndim == 1:
            waveform = torch.tensor(waveform).unsqueeze(0)
        else:
            waveform = torch.tensor(waveform.T)

        meta = {k: v for k, v in sample.items() if k != "audio"}
        return waveform, sr, meta


def collate_batch(batch):
    """Pads variable-length audio for batching"""
    waveforms, srs, metas = zip(*batch)
    srs = torch.tensor(srs)
    max_len = max(w.shape[1] for w in waveforms)
    padded = [torch.nn.functional.pad(w, (0, max_len - w.shape[1])) for w in waveforms]
    waveforms = torch.stack(padded)
    return waveforms, srs, metas

In [ ]:
# Load dataset
ds = load_dataset("ARTPARK-IISc/vaani", "Goa_NorthSouthGoa", split="train")

# Disable auto-decoding to get raw bytes
ds = ds.cast_column("audio", Features({"path": Value("string"), "bytes": Value("binary")}))

#filter only hindi samples
ds = ds.filter(lambda x: x["language"] == "Hindi")

print("Hindi samples:", len(ds))

# Wrap dataset
wrapped_ds = HFDataWrapper(ds)

# Create DataLoader
loader = TorchLoader(wrapped_ds, batch_size=4, shuffle= False, collate_fn=collate_batch)

print("✅ DataLoader ready")

Using the latest cached version of the dataset since ARTPARK-IISc/vaani couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'Goa_NorthSouthGoa' at C:\Users\aadit\.cache\huggingface\datasets\ARTPARK-IISc___vaani\Goa_NorthSouthGoa\0.0.0\dcb8ea6dd7050ec2ffb5e07542a33da7bef2a8fc (last modified on Fri Nov 14 14:09:49 2025).


Loading dataset shards:   0%|          | 0/44 [00:00<?, ?it/s]

Hindi samples: 53761
✅ DataLoader ready


In [ ]:
class ASRPipeline:
    def __init__(self, processor, model, device=None):
        self.processor = processor
        self.model = model
        self.device = device or "cpu"
        self.model.to(self.device)

        # Force Hindi transcription
        self.model.generation_config.task = "transcribe"
        self.model.generation_config.language = "hi"

        # Also fix the forced decoder IDs
        forced_ids = self.processor.get_decoder_prompt_ids(
            task="transcribe",
            language="hi"
        )
        self.model.generation_config.forced_decoder_ids = forced_ids

    def transcribe_batch(self, waveforms, sample_rates):
        # waveforms shape: (batch, 1, time)
        if isinstance(waveforms, torch.Tensor):
            if waveforms.ndim == 3:
                waveforms = waveforms.squeeze(1)  # (batch, time)
            elif waveforms.ndim == 2:
                pass  # already (batch, time)
            elif waveforms.ndim == 1:
                waveforms = waveforms.unsqueeze(0)
            else:
                raise ValueError(f"Unexpected waveform shape: {waveforms.shape}")
        else:
            raise ValueError("waveforms must be a torch.Tensor")

        # Convert each waveform to numpy
        waveforms = [w.cpu().numpy() for w in waveforms]

        # Extract sample rate
        if isinstance(sample_rates, (list, tuple, torch.Tensor)):
            sr = int(sample_rates[0])
        else:
            sr = int(sample_rates)

        # Extract mel spectrograms
        inputs = self.processor.feature_extractor(
            waveforms,
            sampling_rate=sr,
            return_tensors="pt",
            padding="longest"
        )

        input_features = inputs.input_features  # (batch, 80, frames)

        B, n_mels, frames = input_features.shape
        target_len = 3000

        if frames < target_len:
            pad_amt = target_len - frames
            input_features = torch.nn.functional.pad(input_features, (0, pad_amt))
        elif frames > target_len:
            input_features = input_features[..., :target_len]

        input_features = input_features.to(self.device)

        with torch.no_grad():
            predicted_ids = self.model.generate(
                input_features=input_features,
                task="transcribe",
                language="hi",
            )

        # Decode text
        transcriptions = self.processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )

        return transcriptions

In [5]:
model_name = "openai/whisper-small"

processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

asr = ASRPipeline(processor=processor, model=model, device="cpu")
print("✅ ASR Pipeline loaded on CPU")

✅ ASR Pipeline loaded on CPU


In [8]:

# Limit to a few batches so it doesn’t spam your notebook
for batch_idx, (waveforms, srs, metas) in enumerate(loader):
    transcriptions = asr.transcribe_batch(waveforms, srs)

    print(f"\n🎯 Batch {batch_idx + 1}\n{'-' * 60}")
    for i, (meta, text, waveform) in enumerate(zip(metas, transcriptions, waveforms)):
        print(f"\n[{i + 1}] Language: {meta['language']}")
        print(f"Gender: {meta.get('gender', 'Unknown')}")
        print(f"District: {meta.get('district', 'N/A')}")
        print(f"Transcription: {text}")

        # Convert waveform to 1D NumPy array for playback
        audio_np = waveform.squeeze().cpu().numpy()
        display(Audio(audio_np, rate=int(srs[0])))

    # only 1 batch to check transcription vs audio
    if batch_idx == 0:
        break


🎯 Batch 1
------------------------------------------------------------

[1] Language: Hindi
Gender: Female
District: NorthSouthGoa
Transcription:  अपा



[2] Language: Hindi
Gender: Female
District: NorthSouthGoa
Transcription:  अप आप बद़ बद़ जाहाज़ लगा आप यह ती शुच्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्च्



[3] Language: Hindi
Gender: Female
District: NorthSouthGoa
Transcription:  जो की राूंशेब में लगी हूँँई हैं



[4] Language: Hindi
Gender: Female
District: NorthSouthGoa
Transcription:  जाली की किड़किया है उपर ग्रीन दिवार है
